In [14]:
import pandas as pd
import xgboost as xgb
import numpy as np
from numpy import argmax
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [15]:
# Read data
dataset = pd.read_csv('C:/Users/kosta/Documents/Industry_4.0/PredictionModelPrototype/MarketPriceData.csv', delimiter=",")

# Transform all categorical values to integers
for key,value in dataset.iteritems():
        #for key in dataset.columns:
            if dataset[key].dtype == type(object):
                le = LabelEncoder()
                dataset[key] = le.fit_transform(dataset[key].astype(str))


In [16]:

# Check and replace missing values with the mean of each column
for key,value in dataset.iteritems():
    for key in dataset.columns:
            if dataset[key].isnull().values.any():
             dataset[key] =  dataset[key].fillna(dataset[key].mean())
            
# In case price_fixed has NaNs replace them with the mean of SMP(MW/h)
if dataset["PRICE_FIXED"].isnull().values.any():
        dataset[key] =  dataset[key].fillna(dataset['SMP(€/MWH)'].mean())
        


In [17]:
# All columns with the last as X, and last column for Y
X = np.array(dataset.iloc[:,:-1])
y = np.array(dataset.iloc[:,-1])

# Convert the dataset into an optimized data structure
data_dmatrix = xgb.DMatrix(data=X,label=y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [18]:
# K - Fold Cross validation


#--------- FIRST ATTEMPT----------#
# equal weight assignment over provider fixed classes
"""weight = 100 / len(dataset['PRICE_FIXED'].unique())

model = xgb.XGBClassifier(objective ='multi:softprob', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 3, alpha = 10, n_estimators = 500, early_stopping_rounds=10,
                          sample_weight= weight, num_class = 3)

kfold = StratifiedKFold(n_splits=5, random_state=7)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))"""

#--------SECOND ATTEMPT---------
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(X):   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    xgb_model = xgb.XGBRegressor(objective="reg:linear")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))
    
#score an acc. percentage
print(np.sqrt(scores))
print("Accuracy: %.2f%% (%.2f%%)" % (np.sqrt(scores).mean()*100, np.sqrt(scores).std()*100))

[22:32:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.00151474 0.00151092 0.00151092 0.00151092 0.00151092]
Accuracy: 0.15% (0.00%)


In [19]:
#model.fit(X_train,y_train)
xgb_model.fit(X_train,y_train)

[22:32:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)